In [3]:
import geopandas as gpd
from tqdm import tqdm

input_file = "../_data/AusUrbHI HVI data unprocessed/NHSD/nhsd_nov2020_utf8.shp"
# note, the study area file is already buffered by 800m using arcgis pro
study_area_file = "../_data/AusUrbHI HVI data unprocessed/NHSD/ausurbhi_study_area_2021.shp"
output_file = "../_data/AusUrbHI HVI data processed/NHSD/gp_ed_within_800m.shp"

nhsd_df = gpd.read_file(input_file)
study_area_df = gpd.read_file(study_area_file)

# create sa1 area dict
sa1_area_dict = study_area_df.set_index('SA1_CODE21')['AREASQKM21'].to_dict()

# Create a new DataFrame for output
new_df = study_area_df[['SA1_CODE21', 'geometry', 'AREASQKM21']].copy()

# Ensure both are in the GDA94 CRS
gda94_crs = 'EPSG:4283'
if nhsd_df.crs != gda94_crs:
    nhsd_df = nhsd_df.to_crs(gda94_crs)
if study_area_df.crs != gda94_crs:
    study_area_df = study_area_df.to_crs(gda94_crs)

In [5]:
# Count the points within the buffer
gp_counts = []
ed_counts = []

for area in tqdm(study_area_df.geometry,
                 desc=f"Counting services",
                 total=len(study_area_df)):
    
    gp_count = nhsd_df[nhsd_df['snomed_nm'] == 'General practice service'].within(area).sum()
    gp_counts.append(gp_count)
    
    ed_count = nhsd_df[nhsd_df['snomed_nm'] == 'Emergency department service'].within(area).sum()
    ed_counts.append(ed_count)

Counting services: 100%|██████████| 13537/13537 [22:36<00:00,  9.98it/s] 


In [6]:
new_df['gp_in_800m'] = gp_counts
new_df['ed_in_800m'] = ed_counts
new_df['gp_density'] = new_df['gp_in_800m'] / new_df['AREASQKM21']
new_df['ed_density'] = new_df['ed_in_800m'] / new_df['AREASQKM21']

print(new_df.head())

    SA1_CODE21                                           geometry  AREASQKM21  \
0  10201102801  POLYGON ((151.41763 -33.45317, 151.41781 -33.4...      0.6349   
1  10201102802  POLYGON ((151.42936 -33.45715, 151.42946 -33.4...      1.1648   
2  10201102803  POLYGON ((151.42585 -33.45684, 151.42621 -33.4...      0.5739   
3  10201102804  POLYGON ((151.41553 -33.47173, 151.41553 -33.4...      0.1519   
4  10201102805  POLYGON ((151.43237 -33.47484, 151.43232 -33.4...      0.1310   

   gp_in_800m  ed_in_800m  gp_density  ed_density  
0           1           0    1.575051         0.0  
1           1           0    0.858516         0.0  
2           1           0    1.742464         0.0  
3           1           0    6.583278         0.0  
4           0           0    0.000000         0.0  


In [7]:
# Restoring the original geometry for SA1 areas (without the buffer)
study_area_df = gpd.read_file('../_data/study area/ausurbhi_study_area_2021.shp')
new_df['geometry'] = study_area_df['geometry']

new_df.to_file(output_file)